<a href="https://colab.research.google.com/github/Nohyeong/PFAS_ML/blob/main/AFFF_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

!pip install rdkit

In [17]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, MolFromSmiles
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from transformers import AutoTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import hyperopt
from hyperopt import STATUS_OK


def positional_encoding(length, depth):

  positions = np.arange(length)[:, np.newaxis]
  depths = np.arange(depth)[np.newaxis, :]/depth

  angle_rates = 1 / (10000**depths)
  angle_rads = positions * angle_rates

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  pos_encoding = pos_encoding[:, :depth]

  return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, d_model):
    super().__init__()
    self.d_model = d_model
    self.dense = tf.keras.layers.Dense(d_model)
    self.pos_encoding = positional_encoding(length=d_model, depth=d_model)

  def call(self, x):
    length = tf.shape(x)[1]
    x = tf.expand_dims(x, -1)
    x = self.dense(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[:length, :]
    return x

class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, FF_num_layers, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential()
    for _ in range(int(FF_num_layers)):
        self.seq.add(tf.keras.layers.Dense(dff, activation='gelu'))
        self.seq.add(tf.keras.layers.Dropout(dropout_rate))
        dff = dff // 2
    self.seq.add(tf.keras.layers.Dense(d_model))
    self.linear = tf.keras.layers.Dense(d_model)
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    seq_output = self.seq(x)
    x = self.add([x, seq_output])
    x = self.linear(x)
    x = self.layer_norm(x)
    return x

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff, dropout_rate)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(d_model=d_model, )

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    x = self.pos_embedding(x)
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)
    return x

class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(1, activation='linear')

  @tf.function(reduce_retracing=True)
  def call(self, inputs):
    x = inputs
    x = self.encoder(x)
    x = GlobalMaxPooling1D()(x)
    pred = self.final_layer(x)
    return pred

def scale_inputs(df, columns):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[columns])
    return scaled_data, scaler

def preprocess_smiles(smiles_list):
    smiles_list = smiles_list.to_list()
    d_model = 64

    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(smiles_list)
    sequences = tokenizer.texts_to_sequences(smiles_list)
    max_length = max([len(seq) for seq in sequences])
    padded_sequences = pad_sequences(sequences, maxlen=56, padding='post')
    return padded_sequences

def hyp_opt(params):
    print('hyp; params: ', params)
    tf.get_logger().setLevel('ERROR')
    num_heads = params['num_heads']
    num_layers = params['num_layers']
    learning_rate = params['learning_rate']
    dff = params['dff']
    dropout_rate = params['dropout_rate']
    batch_size = params['batch_size']
    epochs = params['epochs']
    d_model = params['d_model']

    mse_fold = []

    X_train = train.copy()
    y_train = train['Removal_rate']

    train_losses = []
    val_losses = []

    encoded_molecular_data = preprocess_smiles(X_train['SMILES'])
    encoded_molecular_data = pd.DataFrame(encoded_molecular_data)

    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)

    train_scaled_inputs, scaler = scale_inputs(train, input_vars)
    X_train_concate = pd.concat([pd.DataFrame(train_scaled_inputs), encoded_molecular_data], axis=1)

    splitter = GroupKFold(n_splits=5)

    best_params = None
    best_RMSE = float('inf')
    best_MAE = float('inf')

    cv_rmse = []
    cv_mae = []

    for train_index, val_index in splitter.split(X_train, y_train, groups=train['group']):


        X_train_fold, X_val_fold = tf.gather(X_train_concate, train_index), tf.gather(X_train_concate, val_index)
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        inputs = Input(shape=(d_model,))

        transformer = Transformer(num_layers=num_layers, d_model=d_model, num_heads=num_heads,
                                  dff=dff, dropout_rate=dropout_rate)

        outputs = transformer(inputs)

        transformer.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

        history = transformer.fit(X_train_fold, y_train_fold,
                                  validation_data=(X_val_fold, y_val_fold),
                                  batch_size = batch_size,
                                  epochs=epochs, verbose=0)

        train_losses.append(history.history['loss'])
        val_losses.append(history.history['val_loss'])

        y_val_pred = transformer.predict(X_val_fold)
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_val_pred))
        mae = mean_absolute_error(y_val_fold, y_val_pred)

        cv_rmse.append(rmse)
        cv_mae.append(mae)

    rmse_score = np.mean(cv_rmse)
    mae_score = np.mean(cv_mae)

    avg_train_loss = np.mean(train_losses, axis=0)
    avg_val_loss = np.mean(val_losses, axis=0)

    index_of_smallest = np.argmin(avg_val_loss)

    return {'loss': rmse_score, 'status': STATUS_OK, 'params': best_params}

def preprocess_data(df):
    df['SMILES'] = df['SMILES'].str.replace('[()=]', '', regex=True)
    df['charge'] = df.apply(lambda row: calculate_charge(row['pH'], row['pKa'], is_acid=True), axis=1)
    df['charge_product'] = (df['IEP'] - df['pH']) * df['charge']
    df['Init_conc'] = df['Init_conc']/df['Mw']
    df['group'] = df['ref_ID'].map(str) + df['PFAS']

    return df

def calculate_charge(pH, pKa, is_acid=True):
    ratio = 10 ** (pH - pKa)
    fraction_deprotonated = ratio / (1 + ratio)

    if is_acid:
        charge = -fraction_deprotonated
    else:
        charge = 1 - fraction_deprotonated

    return charge

In [18]:
df = pd.read_excel('AFFF_data_final_20240308.xlsx', sheet_name='data')

df.columns = ['ID', 'Membrane', 'MWCO', 'IEP',
              'CA', 'PFAS', 'Mw', 'Atomic charge', 'SMILES',
              'Size', 'log Kow', 'pKa', 'Init_conc',
              'IS', 'Pres', 'pH', 'Removal_rate', 'ref_ID', 'ref']

df = preprocess_data(df)
df = df.dropna(subset=['IEP'])
df.index = range(len(df))

input_vars = ['MWCO', 'CA', 'Size', 'log Kow', 'Init_conc', 'Pres', 'IS', 'charge_product']

train = df[df['ref_ID']!=1]
test = df[df['ref_ID']==1]

In [19]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from tensorflow.keras.utils import pad_sequences
import matplotlib.pyplot as plt

d_model = 64
num_heads_values = [2, 4, 8, 16, 32]
dff_values = [4, 8, 16, 32]
num_layers_values = [1, 2, 3, 4, 5, 6, 7, 8]
batch_size_values = [4, 8, 16, 32, 64]
FF_num_layers_values = [1, 2, 3, 4, 5, 6, 7, 8]

space = {'num_heads': hp.choice('num_heads', num_heads_values),
         'num_layers': hp.choice('num_layers', num_layers_values),
         'dff': hp.choice('dff', dff_values),
         'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.3),
         'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
         'batch_size': hp.choice('batch_size', batch_size_values),
         'epochs': hp.uniformint('epochs', 1, 500),
         'FF_num_layers': hp.choice('FF_num_layers', FF_num_layers_values),
         'd_model': d_model}

trials = Trials()
best_params = fmin(fn=hyp_opt, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

best_params['num_heads'] = num_heads_values[best_params['num_heads']]
best_params['num_layers'] = num_layers_values[best_params['num_layers']]
best_params['dff'] = dff_values[best_params['dff']]
best_params['batch_size'] = batch_size_values[best_params['batch_size']]
best_params['FF_num_layers'] = FF_num_layers_values[best_params['FF_num_layers']]

In [20]:
from datetime import datetime

encoded_molecular_data = pd.DataFrame(preprocess_smiles(train['SMILES']))
d_model = 64

train_scaled_inputs, scaler = scale_inputs(train, input_vars)
X_train_concate = pd.concat([pd.DataFrame(train_scaled_inputs), encoded_molecular_data], axis=1)
X_train_tf = tf.convert_to_tensor(X_train_concate)
y_train = train['Removal_rate']

inputs = Input(shape=(d_model,))

transformer = Transformer(num_layers=best_params['num_layers'], d_model=d_model, num_heads=
                          best_params['num_heads'], dff=best_params['dff'],
                          dropout_rate=best_params['dropout_rate'])

outputs = transformer(inputs)

transformer.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mse')

history = transformer.fit(X_train_tf, y_train,
                          batch_size = best_params['batch_size'],
                          epochs=int(best_params['epochs']), verbose=1)

X_test_scaled = scaler.fit_transform(test[input_vars])
test_encoded_molecular_data = pd.DataFrame(preprocess_smiles(test['SMILES']))

test.index = range(len(test))

X_test_concate = pd.concat([pd.DataFrame(X_test_scaled), test_encoded_molecular_data], axis=1)
X_test_tf = tf.convert_to_tensor(X_test_concate)
y_test = test['Removal_rate']

y_pred = transformer.predict(X_test_tf)
test['y_pred'] = y_pred
train_predictions = transformer.predict(X_train_tf)
train['y_pred'] = train_predictions

In [21]:
import plotly.express as px
import plotly.graph_objects as go

def plot_predictions(train, test, train_predictions):
    train['mark'] = 'train'
    test['mark'] = 'test'
    total_data = pd.concat([train, test])

    X_title = 'Real removal (%)'
    Y_title = 'Pred removal (%)'
    title_font_size = 43
    tickfont_size = 35
    ticklen = 5
    tickwidth = 2
    title_standoff = 20
    plot_width = 600
    plot_height = 500

    train_pred_mae = mean_absolute_error(train_predictions, train['Removal_rate'])
    train_pred_rmse = mean_squared_error(train_predictions, train['Removal_rate'], squared=False)

    pred_mae = mean_absolute_error(test['y_pred'], test['Removal_rate'])
    pred_rmse = mean_squared_error(test['y_pred'], test['Removal_rate'], squared=False)

    print(f"MAE: {np.round(pred_mae,2)}, RMSE: {np.round(pred_rmse,2)}")
    print(f"Train MAE: {np.round(train_pred_mae,2)}, Train RMSE: {np.round(train_pred_rmse,2)}")


    fig = px.scatter(total_data, x='Removal_rate', y='y_pred', color='mark', width=plot_width, height=plot_height, range_x=[-10, 120], range_y=[-10, 120])

    # Customize layout
    fig.update_layout(
        xaxis=dict(
            dtick=25,
            title_text=X_title,
            title_font={"size": title_font_size},
            tickfont=dict(family='Arial', color='black', size=tickfont_size),
            title_standoff=title_standoff,
            color='black',
            ticks="inside",
            ticklen=ticklen,
            tickwidth=tickwidth,
            title_font_family="Arial"),
        yaxis=dict(
            dtick=25,
            title_text=Y_title,
            title_font={"size": title_font_size},
            tickfont=dict(family='Arial', color='black', size=tickfont_size),
            title_standoff=title_standoff,
            color='black',
            ticks="inside",
            ticklen=ticklen,
            tickwidth=tickwidth,
            title_font_family="Arial"))

    fig.update_layout(plot_bgcolor='rgb(256,256,256)', showlegend=False)

    fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)

    random_x = [0, 100]
    random_y0 = [0, 100]
    fig.add_trace(go.Scatter(x=random_x, y=random_y0, line=dict(color='red', width=2), marker=dict(size=1)))

    fig.update_traces(marker=dict(size=10, opacity=0.65, line=dict(width=0.7)), selector=dict(mode='markers'))

    fig.show()

plot_predictions(train, test, train_predictions)